In [ ]:
class OscGenProcess: 
    def __init__(self, amplitude, frequency, phase_shift, duration):
        self.amplitude = amplitude
        self.frequency = frequency
        self.phi = phi
        self.duration = duration
        self.times = []
        self.outputs = []

    def generate_sine_wave(self):
        # Calculate the period and interval between samples
        period = 1 / self.frequency
        interval = period / 10

        # Initialize timing
        start_time = time.time()
        end_time = start_time + self.duration

        # Real-time oscillator loop
        while time.time() < end_time:
            current_time = time.time() - start_time
            omega = 2 * np.pi * self.frequency
            y = self.amplitude * np.sin(omega * current_time + self.phase_shift)
            
            # Append current time and output to the lists, converting time to milliseconds
            self.times.append(current_time * 1000)
            self.outputs.append(y)
            
            # Sleep until the next sample
            next_sample_time = time.time() + interval
            while time.time() < next_sample_time:
                pass

    def plot_results(self):
        # Plotting the results
        plt.figure(figsize=(25, 5))
        plt.plot(self.times, self.outputs, label='Sine Wave')
        plt.title('Sine Wave Output Over Time')
        plt.xlabel('Time (milliseconds)')
        plt.ylabel('Amplitude')
        plt.grid(True)
        plt.legend()
        plt.show()    

'''
Usage: 
oscillator = OscGenProcess(amplitude=50, frequency=500, phase_shift=0, duration=0.1)
oscillator.generate_sine_wave()
oscillator.plot_results()
print("Finished generating the sine wave.")
'''